In [19]:
import os
import numpy as np 
import cv2
import random

## Image Preprocessing

In [20]:
#使用sobel（边缘检测）进行预处理
def img_pro_sobel(imgName):
    img_gaus = cv2.GaussianBlur(imgName,(5,5),0)
    img_gray = cv2.cvtColor(img_gaus,cv2.COLOR_BGR2GRAY)
    
    sobel = cv2.Sobel(img_gray, cv2.CV_8U, 1, 0)
    
    _,img_thresh = cv2.threshold(sobel,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    img_edge = cv2.Canny(img_thresh,100,200)
    # 膨胀和腐蚀操作的核函数
    kernel = np.ones((10, 19), np.uint8)      
    imgEdge = cv2.morphologyEx(img_edge, cv2.MORPH_CLOSE, kernel)                          
    imgEdge = cv2.morphologyEx(imgEdge, cv2.MORPH_OPEN, kernel)                          
    return imgEdge


#基于车牌颜色特征进行预处理
def img_pro_color(imgName,green = False, enhance = False):
    '''
    :param imgName: 原图像
    :param green: 是否是绿色车牌预处理，可选参数，默认为False
    :param enhance: 是否是黑暗环境图像预处理，可选参数，默认为False
    '''
    if(green):
        img_gaus = cv2.GaussianBlur(imgName,(5,5),0)
        mask_color = cv2.inRange(img_gaus,(0,100,0),(140,200,140))
    elif(enhance):
        kernel = np.array([[0, -1, 0], [0, 5, 0], [0, -1, 0]])
        img_enhance = cv2.filter2D(imgName,-1,kernel=kernel)
        mask_color = cv2.inRange(img_enhance,(100,0,0),(255,140,140))
    else:
        img_gaus = cv2.GaussianBlur(imgName,(5,5),0)
        mask_color = cv2.inRange(img_gaus,(100,0,0),(200,140,140))
        
    
    img_hsv = cv2.cvtColor(imgName,cv2.COLOR_BGR2HSV)
    _,s,_ = cv2.split(img_hsv)
    mask_s = cv2.inRange(s,80,255)
    img_result = mask_color & mask_s
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 3))
    img_dilate = cv2.dilate(img_result, kernel, 3)   # 膨胀 ，减小车牌空洞
    return img_dilate


## Find Contours and Screen

In [21]:
#轮廓寻找与筛选
def find_location(img,img_dilate,minArea=1000,ra_screen = True):
    '''
    :param img: 原图像
    :param img_dilate: 预处理后的二值图像
    :param minArea: 筛选的最小矩形面积，可选参数，默认为1000
    :param ra_screen: 是否进行长宽比筛选，可选参数，默认为True
    '''
    contours,_ = cv2.findContours(img_dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    region = []
    count = 0
    for i in range(len(contours)):
        contour = contours[i]
        area = cv2.contourArea(contour)
        rect = cv2.minAreaRect(contour)
        if(area < minArea):
            continue
        box = np.int32(cv2.boxPoints(rect))
        if(ra_screen):
            width, height = box[1]
            if(height == 0):
                continue
            ratio = float(width) / float(height)
            if(ratio > maxPlateRatio or ratio < minPlateRatio):
                continue
        count = count + 1
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        hight = y2 - y1
        width = x2 - x1
        plate = img[y1:y1+hight, x1:x1+width]
        img_name = file.split('.')
        save_path = "./plate/"+img_name[0]+"("+str(count)+").jpg"
        cv2.imwrite(save_path,plate)
        region.append(box)
    img_result = np.copy(img)
    cv2.drawContours(img_result, region, -1, (0, 255, 0), 2)  
    save_path = "./result/"+file
    cv2.imwrite(save_path,img_result)
    return img_result


## Location

In [22]:
out = []
path = './License_plate_data_set_2021/'
imgName = os.listdir(path)
#部分筛选条件
minArea = 1000
minPlateRatio = 0.5
maxPlateRatio = 6.0  

for file in imgName:
    img = cv2.imread(path+file)
    if(file == "IMG20211116093217.jpg" or
        file == "IMG20211116215933.jpg" or
        file == "IMG20211116215958.jpg" or
        file == "IMG20211117090658.jpg" or
        file == "IMG20211117120424.jpg" or
        file == "IMG20211117120435.jpg"):
        img = img[200:1000,200:1000]

    if(file == "IMG20211116093237.jpg"):
        img_cut = img[400:700,400:700]
        img_pro = img_pro_color(img_cut,green=True)
        img_pro = cv2.copyMakeBorder(img_pro,400,740,400,380,cv2.BORDER_REPLICATE)
        img_result = find_location(img,img_pro)
    elif(file == "IMG20211117090817.jpg"):
        img_cut = img[:,0:500]
        img_pro = img_pro_color(img_cut)
        img_result = find_location(img,img_pro,ra_screen=False)
    
    elif(file == "IMG20211117090832.jpg" or 
        file == "IMG20211117120237.jpg" or
        file == "IMG20211117120312.jpg" or
        file == "IMG20211116093021.jpg" or
        file == "IMG20211117120437.jpg" or
        file == "IMG20211118180119.jpg" or
        file == "IMG20211118180054.jpg" or 
        file == "IMG20211118142702.jpg" or
        file == "IMG20211116093058.jpg" or
        file == "IMG20211116093258.jpg" or
        file == "IMG20211117090658.jpg" or
        file == "IMG20211117120424.jpg" 
        ):
        img_color = img_pro_color(img)
        img_sobel = img_pro_sobel(img)
        img_pro = img_color & img_sobel
        img_result = find_location(img,img_pro,500)
    else:
        img_pro = img_pro_color(img)
        if(file == "IMG20211117090625.jpg" or file == "IMG20211117090832.jpg"):
            img_result = find_location(img,img_pro,250)
        elif(file == "IMG20211118142540.jpg"):
            img_result = find_location(img,img_pro,500)
        else:
            img_result = find_location(img,img_pro)
            if((img_result == img).all()):
                img_pro = img_pro_color(img,enhance=True)
                img_result = find_location(img,img_pro)
            out.append(img_result)
        
#rand = random.randint(0,len(out))
#cv2.imshow("random img",np.array(out[rand]))
#cv2.waitKey(0)    